# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:
salaries = pd.read_csv('../Data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [26]:
sample = salaries[salaries['Hourly Rate'].isna() == False]['Hourly Rate']
sample_mean = sample.mean()
print(sample.mean())

st.ttest_1samp(sample,30)

32.78855771628024


TtestResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92, df=8021)

In [24]:
sample

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [5]:
st.ttest_1samp(sample,30, alternative= 'greater')

TtestResult(statistic=20.6198057854942, pvalue=2.1615120243114947e-92, df=8021)

In [6]:
st.ttest_1samp(sample,30, alternative= 'less')

TtestResult(statistic=20.6198057854942, pvalue=1.0, df=8021)

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [27]:
from scipy import stats
# sample mean
sample_mean = np.mean(sample)
# standard error of the mean
standard_error = stats.sem(sample)
# confidence interval
confidence_level = 0.95
degrees_of_freedom = len(sample) - 1
confidence_interval = stats.t.interval(
    confidence_level,
    degrees_of_freedom,
    loc=sample_mean,
    scale=standard_error)
confidence_interval

(32.52345834488425, 33.05365708767623)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [34]:
from statsmodels.stats.proportion import proportions_ztest
#Sample proportion
sample_proportion = len(sample) / len(salaries)
sample_proportion

#count of workers working hourly or salary
count_total = len(salaries)

#count of workers working hourly
count = len(sample)

#hypothesised proportion
p0 = 0.25

#performing ztest
stat, p_value = proportions_ztest(count, count_total, value= p0)
print(stat, p_value)

alpha = 0.05
if p_value < alpha:
    print("Reject the null hypotesis")
else:
    print("Do not reject the null hypotesis")


-3.5099964213703005 0.0004481127249057967
Reject the null hypotesis
